In [1]:
%reload_ext autoreload
%autoreload 2

import nest_asyncio
nest_asyncio.apply()

In [2]:
import phoenix as px
from phoenix.trace.langchain import LangChainInstrumentor

# start the phoenix app
session = px.launch_app()
# Initialize Langchain auto-instrumentation
LangChainInstrumentor().instrument()

/home/jjmachan/.pyenv/versions/3.10.12/envs/nirant-class/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
I0000 00:00:1726327324.950860   15981 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [6]:
from langchain_openai import ChatOpenAI
from ragas.llms import LangchainLLMWrapper

llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [32]:
def process_row(row, correct=False, column="response"):
    if correct:
        row[column] = row["correct"]
    else:
        row[column] = row["incorrect"]
    return row

### Faithfulness

In [19]:
row = {
    "user_input": "Where and when was Einstein born?",
    "retrieved_contexts": ["Albert Einstein (born 14 March 1879) was a German-born theoretical physicist, widely held to be one of the greatest and most influential scientists of all time"],
    "correct": "Einstein was born in Germany on 14th March 1879.",
    "incorrect": "Einstein was born in Germany on 20th March 1879.",
}

from ragas.metrics import faithfulness
faithfulness.llm = judge_llm

faithfulness.score(process_row(row, correct=False))

/tmp/ipykernel_1784025/361441268.py:11: DeprecationWarning: The function score was deprecated in 0.2, and will be removed in the 0.3 release. Use single_turn_ascore instead.
  faithfulness.score(process_row(row, correct=False))


0.5

### Answer Correctness

In [29]:
row = {
    "user_input": "Where and when was Einstein born?",
    "reference": "Einstein was born in 1879 in Germany.",
    "correct": "In 1879, Einstein was born in Germany.",
    "incorrect": "Einstein was born in Spain in 1879.",
}

from ragas.metrics import answer_correctness, answer_similarity
from ragas.embeddings import embedding_factory

answer_correctness.llm = judge_llm
answer_similarity.embeddings = embedding_factory("text-embedding-3-small")
answer_correctness.answer_similarity = answer_similarity

In [31]:
answer_correctness.score(process_row(row, correct=True))

/tmp/ipykernel_1784025/1311459674.py:1: DeprecationWarning: The function score was deprecated in 0.2, and will be removed in the 0.3 release. Use single_turn_ascore instead.
  answer_correctness.score(process_row(row, correct=True))


0.9837395642029906

### Context Recall

In [43]:
row = {
    "user_input": "Where and when was Einstein born?",
    "reference": "Einstein was born in 1879 in Germany.",
    "correct": ["Albert Einstein was born on March 14, 1879 in Ulm, Württemberg, Germany"],
    "incorrect": ["Einstein was born in Ulm, but his family moved to Munich when he was just six weeks old", 
                  "Einstein's birth was registered at the registry office in Ulm on March 15, 1879, the day after he was born"
                  "At the time of Einstein's birth, Ulm was a growing town of about 33,000 inhabitants"],
}

from ragas.metrics import context_recall
context_recall.llm = judge_llm

context_recall._required_columns

('user_input', 'retrieved_contexts', 'reference')

In [44]:
context_recall.score(process_row(row, correct=False, column="retrieved_contexts"))

/tmp/ipykernel_1784025/420289945.py:1: DeprecationWarning: The function score was deprecated in 0.2, and will be removed in the 0.3 release. Use single_turn_ascore instead.
  context_recall.score(process_row(row, correct=False, column="retrieved_contexts"))


0.0

### Context Enity Recall

In [51]:
from ragas.metrics import context_entity_recall
context_entity_recall.llm = judge_llm
context_entity_recall._required_columns
context_entity_recall.score(process_row(row, correct=False, column="retrieved_contexts"))

/tmp/ipykernel_1784025/363734810.py:4: DeprecationWarning: The function score was deprecated in 0.2, and will be removed in the 0.3 release. Use single_turn_ascore instead.
  context_entity_recall.score(process_row(row, correct=False, column="retrieved_contexts"))


0.3333333322222222

### Context Precision

In [37]:
row = {
    "user_input": "Where and when was Einstein born?",
    "reference": "Einstein was born in 1879 in Germany.",
    "correct": ["Albert Einstein was born on March 14, 1879 in Ulm, Württemberg, Germany"],
    "incorrect": ["Einstein was born in Ulm, but his family moved to Munich when he was just six weeks old", 
                  "Einstein's birth was registered at the registry office in Ulm on March 15, 1879, the day after he was born"],
}

from ragas.metrics import context_precision
context_precision.llm = judge_llm
context_precision._required_columns

('user_input', 'retrieved_contexts', 'reference')

In [39]:
context_precision.score(process_row(row, correct=True, column="retrieved_contexts"))

/tmp/ipykernel_1784025/295053778.py:1: DeprecationWarning: The function score was deprecated in 0.2, and will be removed in the 0.3 release. Use single_turn_ascore instead.
  context_precision.score(process_row(row, correct=True, column="retrieved_contexts"))


0.9999999999

### Rubric Based Metrics

In [63]:
from ragas import evaluate
from datasets import Dataset, DatasetDict

from ragas.metrics import labelled_rubrics_score, reference_free_rubrics_score

rows = {
    "user_input": [
        "What's the longest river in the world?",
    ],
    "reference": [
        "The Nile is a major north-flowing river in northeastern Africa.",
    ],
    "response": [
        "The longest river in the world is the Nile, stretching approximately 6,650 kilometers (4,130 miles) through northeastern Africa, flowing through countries such as Uganda, Sudan, and Egypt before emptying into the Mediterranean Sea. There is some debate about this title, as recent studies suggest the Amazon River could be longer if its longest tributaries are included, potentially extending its length to about 7,000 kilometers (4,350 miles).",
    ],
    "retrieved_contexts": [
        [
            "Scientists debate whether the Amazon or the Nile is the longest river in the world. Traditionally, the Nile is considered longer, but recent information suggests that the Amazon may be longer.",
            "The Nile River was central to the Ancient Egyptians' rise to wealth and power. Since rainfall is almost non-existent in Egypt, the Nile River and its yearly floodwaters offered the people a fertile oasis for rich agriculture.",
            "The world's longest rivers are defined as the longest natural streams whose water flows within a channel, or streambed, with defined banks.",
            "The Amazon River could be considered longer if its longest tributaries are included, potentially extending its length to about 7,000 kilometers."
        ],
    ]
}



dataset = Dataset.from_dict(rows)

result = evaluate(
    dataset,
    metrics=[
        labelled_rubrics_score,
        reference_free_rubrics_score
    ],
)

result.to_pandas()

Evaluating: 100%|██████████| 2/2 [00:02<00:00,  1.49s/it]


,user_input,retrieved_contexts,response,reference,labelled_rubrics_score,reference_free_rubrics_score
0,What's the longest river in the world?,[Scientists debate whether the Amazon or the N...,"The longest river in the world is the Nile, st...",The Nile is a major north-flowing river in nor...,4,4
